Generates a list of subjects that should be included in the single-cell analysis.  This list is based on those subjects we included in the early analyses (which were the subjects Chen provided that we happened to have data for). 

In [16]:
from pathlib import Path

import numpy as np
import pandas as pd
import scipy.io

from keller_zlatic_vnc.data_processing import generate_standard_id_from_matlab_id

## Parameters go here 

In [25]:
# Locations of files Chen provided with the original single cell, closed loop activity
sc_cl_activity_folder = r'/Users/bishopw/Documents/Janelia_Research/Projects/keller_zlatic_vnc/data/extracted_dff_v2/'

a00c_cl_act_data_files = ['A00c_activity_A4.mat', 'A00c_activity_A9.mat']

basin_cl_act_data_files = ['Basin_activity_A4.mat', 'Basin_activity_A9.mat']

handle_cl_act_data_files = ['Handle_activity_A4.mat', 'Handle_activity_A9.mat']

# Specify the file to save the list in
save_file = r'/Volumes/bishoplab/projects/keller_vnc/data/single_cell/subjects.csv'


## See which specimens Chen originally provided activity for in the single cell, closed-loop analysis

In [11]:
type_strings = ['a00c', 'handle', 'basin']
cl_act_subjects = [None]*3
for i, files in enumerate([a00c_cl_act_data_files, handle_cl_act_data_files, basin_cl_act_data_files]):
    
    a4_act = scipy.io.loadmat(Path(sc_cl_activity_folder) / files[0], squeeze_me=True)
    a9_act = scipy.io.loadmat(Path(sc_cl_activity_folder) / files[1], squeeze_me=True)
    
    # Correct mistake in labeling if we need to
    if type_strings[i] == 'basin' or type_strings[i] == 'handle':
        ind = np.argwhere(a4_act['newTransitions'] == '0824L2CL')[1][0]
        a4_act['newTransitions'][ind] = '0824L2-2CL'

    a4_subjects = list(a4_act['newTransitions'])
    a9_subjects = list(a9_act['newTransitions'])
    a4_a9_subjects = list(set(a4_subjects + a9_subjects))
    a4_a9_subjects = [generate_standard_id_from_matlab_id(id) for id in a4_a9_subjects]
    a4_a9_subjects.sort()
    cl_act_subjects[i] = a4_a9_subjects

In [27]:
all_subjects = list(set(cl_act_subjects[0] + cl_act_subjects[1] + cl_act_subjects[2]))
all_subjects.sort()
all_subjcts = pd.DataFrame(all_subjects, columns=['Subject'])

## Save the list of subjects

In [28]:
all_subjcts.to_csv(save_file)